<a href="https://colab.research.google.com/github/Ne0Cheng/gpt-ai-assistant/blob/main/%E5%B0%BC%E6%AD%90%E5%8F%B0%E8%82%A1%E5%B7%B4%E8%8F%B2%E7%89%B9%E7%AD%96%E7%95%A5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **下載歷史資料庫**

In [ ]:
# 下載歷史資料庫
!pip install finlab > log.txt
!pip install ta-lib-bin > log.txt

# **巴菲特選股策略**

In [ ]:
# @title
# 導入必要模組
from finlab import data, backtest
import talib

# 獲取財務數據
當月營收 = data.get("monthly_revenue:當月營收")
revenue_growth_rate = data.get('fundamental_features:營收成長率')
roe = data.get('fundamental_features:ROE稅後')
pb_ratio = data.get('price_earning_ratio:股價淨值比')
volume = data.get('price:成交股數')
close = data.get('price:收盤價')

# 計算月營收斜率
def rev_growth_func(rev):
    return talib.LINEARREG_SLOPE(rev, timeperiod=12) / talib.LINEARREG_INTERCEPT(rev,12)

rev_grouwth = 當月營收.apply(rev_growth_func)

# 定義選擇標準
revenue_growth_threshold = revenue_growth_rate.quantile(0.8, axis=1)
roe_threshold = roe.mean(axis=1)

# 找尋符合標準的股票
revenue_growth_stocks = revenue_growth_rate > revenue_growth_threshold
high_roe_stocks = (roe > roe.quantile(0.5, axis=1)).deadline()
rev_sl_compare = (當月營收.average(3) > 當月營收.average(12)) & (當月營收 > 當月營收.shift(12))

# 定義購買條件
buy_condition = (

    # 把 ROE > 80% 改成 ROE > 50%，ROE 在台股會有回歸效應（ROE高不可持續，反而傷害股價）
    high_roe_stocks

    # 增加月營收成長的部分（台股才有月營收，資料公布時間更快）
    & (當月營收.pct_change().rolling(12).std() < 0.3)
    & rev_sl_compare

    # 把本益比換成股價淨值比（本益比在台股上直接用比較困難）
    & (pb_ratio < 2)

    # 增加流動性
    & (volume > 200000)

    # 回測後發現波動（風險）太大，選擇低風險特性的股票
    & (close.pct_change().rolling(60).std() < 0.025)
)

# 月營收增長最快的股票
position = rev_grouwth[buy_condition].is_largest(5)

# 調整換股時間為月營收截止日
position = position.reindex(當月營收.index_str_to_date().index)

# Simulate positions
result = backtest.sim(position, position_limit=0.2, name='巴菲特選股法優化版')